In [18]:
# !pip install -U langchain-community
# !pip install unstructured
# !pip install -U langchain-openai

In [29]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import os
import json
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

In [30]:
DATA_PATH = "./"

def load_data():
    docs = []
    # Loop through each file in the directory
    for filename in os.listdir(DATA_PATH):
        if filename.endswith(".JSON"):
            file_path = os.path.join(DATA_PATH, filename)
            with open(file_path, 'r') as file:
                json_data = json.load(file)
                json_content = json.dumps(json_data, indent=2)
                doc = Document(page_content=json_content, metadata={"source": file_path})
                docs.append(doc)
    return docs
docs = load_data()
docs[0].page_content

'[\n  {\n    "payer": {\n      "name": "Merchant1z",\n      "virtualAddress": "7557881346@paytm",\n      "mccCode": "5499",\n      "entityType": "ENTITY",\n      "mobileNo": "9077300001",\n      "email": "",\n      "ratingVerified": "TRUE",\n      "seqNo": "1",\n      "identityType": "ACCOUNT",\n      "aadhaarConsent": "N"\n    },\n    "payee": {\n      "name": "",\n      "virtualAddress": "nisha.goyat099-2@okicici",\n      "mccCode": "0000",\n      "entityType": "PERSON",\n      "mobileNo": "",\n      "email": "",\n      "ratingVerified": "TRUE",\n      "seqNo": "1",\n      "identityType": "ACCOUNT"\n    },\n    "txnNote": "Validate Address",\n    "msgType": "ReqValAdd",\n    "txnType": "ReqValAdd",\n    "devices": [\n      {\n        "name": "IP",\n        "value": "127.0.0.1"\n      },\n      {\n        "name": "TYPE",\n        "value": "INT"\n      },\n      {\n        "name": "LOCATION",\n        "value": "Mumbai"\n      }\n    ]\n  },\n  {\n    "apiStatus": {\n      "errorCode": 

In [31]:
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=50,\
  length_function=len,
  add_start_index=True
)
chunks = text_splitter.split_documents(docs)
print(f"Split {len(docs)} documents into {len(chunks)} chunks")

Split 1 documents into 10 chunks


In [32]:
doc = chunks[4]
print(doc.page_content)
print(doc.metadata)

},
    "payee": {
      "name": "",
      "virtualAddress": "mwt000001@psb",
      "mccCode": "0000",
      "entityType": "PERSON",
      "mobileNo": "",
      "email": "",
      "ratingVerified": "TRUE",
      "seqNo": "1",
      "identityType": "ACCOUNT"
    },
    "txnNote": "Validate Address",
    "msgType": "ReqValAdd",
    "txnType": "ReqValAdd",
    "devices": [
      {
        "name": "IP",
        "value": "127.0.0.1"
      },
      {
        "name": "TYPE",
        "value": "INT"
      },
      {
        "name": "LOCATION",
        "value": "Mumbai"
      }
    ]
  },
  {
    "apiStatus": {
      "errorCode": "0",
      "npciErrorCode": "",
      "cbsErrorCode": "",
      "errorDescription": "Success",
      "txnStatus": 2
    },
    "maskName": "Jaspal Bhatti",
    "payeeCode": "",
{'source': './Data.JSON', 'start_index': 3819}


In [35]:
CHROMA_PATH = "chroma"
OPENAI_API_KEY = "sk-proj-V4yPkDwA4442djQ8V6CfT3BlbkFJ4vXg8AFjSFIybnBChpWf"
# # remove any pre-existing dir
# if os.path.exists(CHROMA_PATH):
#     print("Removing existing directory")
#     os.rmdir(CHROMA_PATH)

db = Chroma.from_documents(chunks, OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY), persist_directory=CHROMA_PATH)
db.persist()
print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [36]:
embedding_function = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
db = Chroma(CHROMA_PATH, embedding_function)
res = db.similarity_search("What is the name of the payer with seqNo 1", k=3)
print(res)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}